In [98]:
%pip -q install google-genai

In [99]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [100]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [101]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [102]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [103]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [104]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [105]:
##########################################
# -- Agente 1: Buscador de Informações -- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especialia dar aulas didáticas, para auxiliar professores. A sua tarefa é usar a ferramenta
        de busca do google (google_search) para encontrar os pontos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 6 pontos relevantes, com base na qualidade e confiabilidade do site em que foi encontrado.
        Se um site tiver pouca confiabilidade, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses pontos devem ser relevantes e atuais.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    pontos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return pontos_buscados

In [113]:
################################################
# --- Agente 2: Organizador de ideias --- #
################################################
def agente_organizador(topico, pontos_buscados):
    organizador = Agent(
        name="agente_organizador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um organizador de ideias que auxilia professores, especialista em criar planos de aula de fácil entendimento. Com base na lista de
        pontos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um plano de aula  sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no plano de aula que será elaborado posteriormente.
        """,
        description="Agente organizador de ideias",
        tools=[google_search]
    )

    entrada_do_agente_organizador = f"Tópico:{topico}\npontos buscados: {pontos_buscados}"
    # Executa o agente
    plano_de_aula = call_agent(organizador, entrada_do_agente_organizador)
    return plano_de_aula

In [114]:
######################################
# Agente 3: Redator do Plano de Aula #
######################################
def agente_redator(topico, plano_de_aula):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Professor Criativo e que os alunos adoram, especializado em criar planos de aula de fácil entendimento,
            didático e bastante explicativo sobre qualquer conteúdos.
            O plano de aula deve ser escrito em português brasileiro.
            Utilize o ponto fornecido no plano de aula e os pontos mais relevantes fornecidos e, com base nisso,
            elabore um plano de aula sobre o tema indicado.
            O plano de aula deve ser informativo, com linguagem simples e bastante didático.
            Não esqueça de por as referências usadas para elaboração do plano de aula, ao final dele.
            """,
        description="Agente redator de planos de aula",
        tools=[google_search]
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de aula: {plano_de_resumo}"
    # Executa o agente
    aula = call_agent(redator, entrada_do_agente_redator)
    return aula

In [116]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Olá Professor(a) 🍎! Bem-vindo ao Chatbot de ensino 📚\n💡 Nosso objetivo é auxiliar você criando planos de aula didáticos 💡")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, sobre qual tópico você gostaria de ensinar?: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Então vamos criar um Plano de Aula sobre {topico} 📝")

    pontos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado da Pesquisa Inicial ---\n")
    display(to_markdown(pontos_buscados))
    print("--------------------------------------------------------------")

    plano_de_resumo = agente_organizador(topico, pontos_buscados)
    print("\n--- 📝 Resultado dos Primeiros Passos para Criação do Seu Plano de Aula ---\n")
    display(to_markdown(plano_de_resumo))
    print("--------------------------------------------------------------")

    resumo = agente_redator(topico, plano_de_resumo)
    print("\n--- 📝 Plano de Aula Pronto! ---\n")
    display(to_markdown(resumo))
    print("--------------------------------------------------------------")



Olá Professor(a) 🍎! Bem-vindo ao Chatbot de ensino 📚
💡 Nosso objetivo é auxiliar você criando planos de aula didáticos 💡
❓ Por favor, sobre qual tópico você gostaria de ensinar?: Cabanagem
Maravilha! Então vamos criar um Plano de Aula sobre Cabanagem 📝

--- 📝 Resultado da Pesquisa Inicial ---



> Okay, vamos buscar os pontos mais relevantes e atuais sobre a Cabanagem para auxiliar professores a darem uma aula didática sobre o tema. Farei algumas buscas no Google para encontrar informações de qualidade e confiáveis.
> 
> 
> Com certeza! Aqui estão 6 pontos relevantes sobre a Cabanagem, elaborados para auxiliar professores a darem uma aula didática sobre o tema, considerando a relevância e confiabilidade das fontes:
> 
> 1.  **Contexto Histórico:** A Cabanagem ocorreu no Grão-Pará (região que abrangia os atuais estados do Amazonas, Pará, Amapá, Roraima e Rondônia) entre 1835 e 1840, durante o Período Regencial do Brasil. Este período foi marcado por instabilidade política após a abdicação de Dom Pedro I e antes da ascensão de Dom Pedro II. A região do Grão-Pará, distante do poder central no Rio de Janeiro, enfrentava tensões políticas e econômicas.
> 2.  **Causas:** A revolta foi motivada por uma combinação de fatores, incluindo a extrema pobreza da população local (indígenas, negros, mestiços e brancos pobres), o isolamento político da região, a forte influência portuguesa mesmo após a Independência do Brasil, e a concentração de poder nas mãos de uma elite local que não atendia aos interesses da maioria. Havia também um desejo de maior autonomia em relação ao governo central.
> 3.  **Participantes e Líderes:** A Cabanagem se destacou pela ampla participação popular, envolvendo diferentes setores da sociedade, desde a elite local descontente até a população mais pobre, conhecida como "cabanos" devido às suas habitações simples. Entre os líderes, destacam-se Félix Clemente Malcher, Francisco e Antônio Vinagre, Eduardo Angelim e Batista Campos.
> 4.  **Desenvolvimento:** A revolta teve início com a tomada de Belém pelos cabanos em janeiro de 1835, resultando na morte de autoridades e na proclamação de um governo provisório. Ao longo dos anos, a Cabanagem passou por diferentes fases, com os cabanos chegando a controlar a província em alguns momentos, mas enfrentando a repressão das forças imperiais.
> 5.  **Consequências:** A Cabanagem foi um conflito extremamente violento, resultando na morte de cerca de 30% a 40% da população do Grão-Pará. A revolta não atingiu seus objetivos de transformação social e política, mas evidenciou as tensões sociais, raciais e econômicas do Brasil Imperial, além de impulsionar discussões sobre desigualdade e injustiça social na região amazônica.
> 6.  **Legado:** Apesar de sua derrota, a Cabanagem é lembrada como um importante movimento de resistência popular na história do Brasil, revelando as profundas desigualdades sociais e regionais do país no século XIX. A revolta também contribuiu para a construção da identidade amazônida e inspirou diversas manifestações culturais e artísticas na região.
> 
> Espero que estes pontos ajudem a enriquecer a sua aula sobre a Cabanagem!
> 


--------------------------------------------------------------

--- 📝 Resultado dos Primeiros Passos para Criação do Seu Plano de Aula ---



> Okay, com base nos pontos que você me forneceu, vou usar a ferramenta de busca do Google para criar um plano de aula mais detalhado e relevante sobre a Cabanagem. Isso incluirá informações adicionais e atuais para auxiliar os professores a darem uma aula didática e envolvente sobre o tema.
> 
> 
> Com base nas informações coletadas, o tema mais relevante para um plano de aula sobre a Cabanagem é:
> 
> **Tema:** A Cabanagem: Revolta Popular na Amazônia Imperial
> 
> **Pontos Relevantes:**
> 
> *   **Contexto Histórico:** Período Regencial (1831-1840), instabilidade política, disputas de poder, Grão-Pará (Amazonas, Pará, Amapá, Roraima e Rondônia), distância do poder central, economia extrativista.
> *   **Causas:** Pobreza da população (indígenas, negros, mestiços, brancos pobres), isolamento político, influência portuguesa, concentração de poder na elite local, desigualdade social, racial e econômica, desejo de autonomia.
> *   **Participantes e Líderes:** Ampla participação popular (elite descontente e população pobre - "cabanos"), Félix Clemente Malcher, Francisco e Antônio Vinagre, Eduardo Angelim, Batista Campos, mulheres cabanas, outros líderes populares.
> *   **Desenvolvimento:** Tomada de Belém (1835), governo provisório cabano, fases da revolta, controle da província, repressão das forças imperiais, violência.
> *   **Consequências:** Grande número de mortos (30-40% da população), não atingiu os objetivos de transformação, evidenciou tensões sociais, raciais e econômicas, discussões sobre desigualdade e injustiça social.
> *   **Legado:** Resistência popular, desigualdades sociais e regionais no Brasil, identidade amazônida, manifestações culturais e artísticas, Memorial da Cabanagem em Belém (Oscar Niemeyer).
> 
> **Plano de Aula Detalhado:**
> 
> 1.  **Introdução (15 minutos):**
>     *   Apresentar o tema da Cabanagem como uma importante revolta popular ocorrida no Brasil durante o Período Regencial.
>     *   Contextualizar o Período Regencial como um período de instabilidade política e revoltas em diversas províncias.
>     *   Mencionar a localização da Cabanagem no Grão-Pará e sua importância para a região amazônica.
> 
> 2.  **Contexto Histórico (20 minutos):**
>     *   Explicar a situação política do Brasil após a abdicação de Dom Pedro I e a formação do Período Regencial.
>     *   Abordar as disputas de poder entre diferentes grupos políticos e a descentralização do poder.
>     *   Descrever a situação do Grão-Pará, incluindo sua economia extrativista e sua distância do poder central.
> 
> 3.  **Causas da Cabanagem (25 minutos):**
>     *   Analisar as causas sociais, econômicas e políticas da revolta.
>     *   Discutir a pobreza da população local e a desigualdade social.
>     *   Abordar a questão da influência portuguesa e a concentração de poder nas mãos da elite local.
>     *   Apresentar o desejo de autonomia da região em relação ao governo central.
> 
> 4.  **Participantes e Líderes (20 minutos):**
>     *   Destacar a ampla participação popular na Cabanagem, incluindo diferentes setores da sociedade.
>     *   Apresentar os principais líderes da revolta, como Félix Clemente Malcher, Francisco e Antônio Vinagre, Eduardo Angelim e Batista Campos.
>     *   Mencionar a importância da participação das mulheres na revolta.
> 
> 5.  **Desenvolvimento da Revolta (30 minutos):**
>     *   Descrever o início da revolta com a tomada de Belém pelos cabanos em 1835.
>     *   Explicar as diferentes fases da Cabanagem e as mudanças no governo provisório.
>     *   Abordar a repressão das forças imperiais e a violência durante o conflito.
> 
> 6.  **Consequências e Legado (20 minutos):**
>     *   Analisar as consequências da Cabanagem, incluindo o grande número de mortos e a não obtenção dos objetivos de transformação social.
>     *   Discutir o impacto da revolta nas tensões sociais, raciais e econômicas do Brasil Imperial.
>     *   Apresentar o legado da Cabanagem como um movimento de resistência popular e sua importância para a construção da identidade amazônida.
>     *   Mencionar o Memorial da Cabanagem em Belém como um símbolo da revolta.
> 
> 7.  **Atividades (20 minutos):**
>     *   Promover um debate sobre a importância da Cabanagem na história do Brasil.
>     *   Analisar documentos históricos e imagens da época para aprofundar o conhecimento sobre a revolta.
>     *   Realizar uma atividade de pesquisa sobre a cultura e a arte inspiradas na Cabanagem na região amazônica.
> 
> 8.  **Conclusão (10 minutos):**
>     *   Reafirmar a importância da Cabanagem como um movimento de resistência popular e sua relevância para a história do Brasil.
>     *   Incentivar os alunos a refletirem sobre as desigualdades sociais e regionais que persistem no país e a importância da luta por justiça social.
> 
> Este plano de aula detalhado oferece um roteiro abrangente para os professores abordarem a Cabanagem de forma didática e envolvente, explorando seus aspectos históricos, sociais, políticos e culturais.
> 


--------------------------------------------------------------

--- 📝 Plano de Aula Pronto! ---



> Olá! Com prazer, elaborei um plano de aula detalhado e didático sobre a Cabanagem, perfeito para seus alunos.
> 
> 
> ## Plano de Aula: A Cabanagem – Revolta Popular na Amazônia Imperial
> 
> **Público-alvo:** Alunos do Ensino Fundamental II (8º ou 9º ano) ou Ensino Médio.
> 
> **Duração:** Aproximadamente 4 aulas de 50 minutos cada (ajustável conforme a necessidade).
> 
> **Objetivos:**
> 
> *   Compreender o contexto histórico do Período Regencial e sua relação com a Cabanagem.
> *   Identificar as causas sociais, econômicas e políticas que levaram à revolta.
> *   Analisar os grupos sociais envolvidos na Cabanagem e seus líderes.
> *   Compreender o desenvolvimento e as principais fases da revolta.
> *   Analisar as consequências e o legado da Cabanagem para a região amazônica e para o Brasil.
> *   Estimular o senso crítico e a reflexão sobre as desigualdades sociais e a importância da participação popular na história.
> 
> **Recursos:**
> 
> *   Mapas do Brasil e da região amazônica.
> *   Textos de apoio (livros didáticos, artigos, documentos históricos).
> *   Imagens e vídeos relacionados à Cabanagem.
> *   Projetor (opcional).
> *   Materiais para produção de cartazes ou apresentações (papel, canetas, etc.).
> 
> ### Aula 1: Introdução e Contexto Histórico
> 
> 1.  **Início (10 minutos):**
>     *   Apresente o tema da Cabanagem como uma revolta popular importante no Brasil durante o Período Regencial (1831-1840).
>     *   Explique que a Cabanagem ocorreu no Grão-Pará, região que abrangia os atuais estados do Amazonas, Pará, Amapá, Roraima e Rondônia.
>     *   Questione os alunos sobre o que eles já sabem sobre o Período Regencial e revoltas populares.
> 
> 2.  **Contexto Histórico (20 minutos):**
>     *   Explique a situação política do Brasil após a abdicação de Dom Pedro I, em 1831, e a formação do Período Regencial, marcado por instabilidade política devido à menor idade do herdeiro ao trono, Dom Pedro II.
>     *   Destaque as disputas de poder entre diferentes grupos políticos, como liberais e conservadores, e a descentralização do poder durante a Regência.
>     *   Mencione outras revoltas ocorridas no Período Regencial, como a Farroupilha e a Balaiada.
>     *   Utilize um mapa para mostrar a localização do Grão-Pará e sua distância do poder central no Rio de Janeiro.
>     *   Explique que, após a independência do Brasil em 1822, o Pará aderiu tardiamente ao Império, apenas em 1823, e de forma forçada, sob ocupação militar, aumentando a insatisfação local.
> 
> 3.  **A Economia e a Sociedade do Grão-Pará (20 minutos):**
>     *   Descreva a economia do Grão-Pará como baseada no extrativismo de produtos como a borracha e a castanha-do-brasil.
>     *   Explique que essa economia era marcada pela exploração da mão de obra indígena e pela desigualdade social.
>     *   Destaque a influência portuguesa na região, que mantinha laços comerciais e culturais fortes com Lisboa, mesmo após a Independência do Brasil.
>     *   Comente sobre a grande concentração de poder nas mãos da elite local, formada por grandes proprietários de terras e comerciantes.
> 
> ### Aula 2: As Causas da Cabanagem
> 
> 1.  **Revisão (10 minutos):**
>     *   Relembre os principais pontos da aula anterior, como o contexto histórico e a situação do Grão-Pará.
>     *   Peça aos alunos que citem as principais características do Período Regencial.
> 
> 2.  **Causas Sociais e Econômicas (20 minutos):**
>     *   Analise as causas sociais da Cabanagem, como a pobreza da população local, formada por indígenas, negros, mestiços e brancos pobres.
>     *   Explique que muitos desses grupos viviam em cabanas às margens dos rios, daí o nome "cabanos".
>     *   Destaque a desigualdade social, racial e econômica como um fator determinante para a revolta.
>     *   Comente sobre a insatisfação com o custo de vida, remuneração baixa e a pobreza e miséria na região.
>     *   Aborde a questão da influência portuguesa e a concentração de poder nas mãos da elite local, que dificultavam a ascensão social dos mais pobres.
> 
> 3.  **Causas Políticas (20 minutos):**
>     *   Discuta as causas políticas da Cabanagem, como o isolamento político do Grão-Pará em relação ao governo central.
>     *   Explique que a província era governada por presidentes nomeados pelo Rio de Janeiro, que nem sempre atendiam aos interesses locais.
>     *   Apresente o desejo de autonomia da região em relação ao governo central como um fator importante para a revolta.
>     *   Explique que muitos políticos brasileiros (membros da elite) queriam mais autonomia política, ou seja, poder tomar mais decisões sem a interferência direta do governo central brasileiro.
> 
> ### Aula 3: Os Participantes e o Desenvolvimento da Revolta
> 
> 1.  **Revisão (10 minutos):**
>     *   Relembre as causas sociais, econômicas e políticas da Cabanagem.
>     *   Pergunte aos alunos quais grupos sociais eram os mais afetados pela desigualdade na região.
> 
> 2.  **Participantes e Líderes (20 minutos):**
>     *   Destaque a ampla participação popular na Cabanagem, incluindo indígenas, negros, mestiços, brancos pobres e até mesmo membros da elite descontente.
>     *   Apresente os principais líderes da revolta, como Félix Clemente Malcher, Francisco e Antônio Vinagre, Eduardo Angelim e Batista Campos.
>     *   Explique o papel de cada um desses líderes na Cabanagem e suas diferentes visões sobre os objetivos da revolta.
>     *   Mencione a importância da participação das mulheres na revolta, como organizadoras, combatentes e provedoras de recursos.
> 
> 3.  **Desenvolvimento da Revolta (20 minutos):**
>     *   Descreva o início da revolta com a tomada de Belém pelos cabanos em 1835.
>     *   Explique as diferentes fases da Cabanagem, marcadas por conflitos internos e externos, mudanças no governo provisório e repressão das forças imperiais.
>     *   Aborde a violência durante o conflito, que resultou em um grande número de mortos e na destruição de vilas e cidades.
>     *   Destaque a tomada do poder pelos cabanos, sendo a Cabanagem referida também como a primeira sublevação popular que conseguiu de fato tomar o poder na história do Brasil.
> 
> ### Aula 4: Consequências e Legado
> 
> 1.  **Revisão (10 minutos):**
>     *   Relembre os principais líderes e as fases da Cabanagem.
>     *   Pergunte aos alunos como a revolta afetou a vida da população do Grão-Pará.
> 
> 2.  **Consequências (20 minutos):**
>     *   Analise as consequências da Cabanagem, incluindo o grande número de mortos (estima-se que 30 a 40% da população da província tenha morrido).
>     *   Discuta o impacto da revolta nas tensões sociais, raciais e econômicas do Brasil Imperial.
>     *   Explique por que a Cabanagem não atingiu os objetivos de transformação social, como a abolição da escravidão e a distribuição de terras.
>     *   Destaque as transformações políticas: a revolta teve um impacto significativo nas políticas do império brasileiro, e a insatisfação popular e a revolta dos cabanos chamaram a atenção para a necessidade de reformas e mudanças no governo.
> 
> 3.  **Legado (20 minutos):**
>     *   Apresente o legado da Cabanagem como um movimento de resistência popular e sua importância para a construção da identidade amazônida.
>     *   Mencione o Memorial da Cabanagem em Belém, projetado por Oscar Niemeyer, como um símbolo da revolta.
>     *   Incentive os alunos a refletirem sobre as desigualdades sociais e regionais que persistem no país e a importância da luta por justiça social.
>     *   Comente sobre as manifestações culturais e artísticas inspiradas na Cabanagem na região amazônica.
> 
> **Atividades Complementares:**
> 
> *   Pesquisa sobre a vida e a obra de Oscar Niemeyer e sua relação com o Memorial da Cabanagem.
> *   Análise de letras de músicas e poemas que retratam a Cabanagem.
> *   Elaboração de um mapa conceitual sobre a Cabanagem, relacionando os principais temas abordados em sala de aula.
> *   Visita a museus ou centros culturais que possuam acervos sobre a Cabanagem (se possível).
> *   Produção de um vídeo ou podcast sobre a Cabanagem, com entrevistas, depoimentos e informações relevantes.
> 
> **Avaliação:**
> 
> *   Participação e envolvimento nas atividades em sala de aula.
> *   Produção de textos e trabalhos sobre a Cabanagem.
> *   Apresentação oral sobre temas relacionados à revolta.
> *   Avaliação escrita sobre os principais conceitos e informações abordados no plano de aula.
> 
> **Referências:**
> 
> *   Livros didáticos de História do Brasil.
> *   Artigos e ensaios sobre a Cabanagem disponíveis em bibliotecas e na internet.
> *   Documentários e filmes sobre a história do Brasil Imperial e suas revoltas.
> *   Sítios eletrônicos de instituições de pesquisa e museus.
> 
> Espero que este plano de aula detalhado seja útil para você! Se precisar de mais alguma coisa, é só me avisar.


--------------------------------------------------------------
